### Part 0: Cellular Automata Model On Network Structure


Cellular automata is discrete dynamical system in space and time consisting of a set of scattered cellular in analog space. Every state in the time sequence of cellular in every moment, is synchronize updated by cells around on the last time via same effect rules (local rules).

In the information diffusion process in social networks, cell of cellular automata can be assumed to be a individual member of the network (node), and cellular space can be assumed as the entire network. Cellular state parameters can be described as the individual characteristic parameters of a member state, cellular automata neighbors represents the range of individual members interaction.

For the purpose of modelling information diffusion, we define the following 3 states:

Healthy: Person yet to come in contact with the information/ rumour

Transmission: Person who believes the rumour

Immune: Person who does not believe the rumour but might become a transmission node if contacted by multiple transmission nodes 


import networkx as nx
import matplotlib as plt
import numpy as np
import copy

In [13]:
# Generation of random graphs
g_ws = nx.watts_strogatz_graph(n = 4000, k = 57,p=0.5) # Watts–Strogatz Graph
g_bs = nx.barabasi_albert_graph(n = 4000, m = 6) # Barabási–Albert Graph

In [14]:
# States
HEALTH = 0
TRANSMISSION = 1
IMMUNE = 2 # immune & inactive

# Parameter
p1 = 0.5 # Contact rate between healthy node and the transmission node
p2 = 0.55 #  Probability that each immune node will become a transmission node
# a = 0.45 # user growth rate per month
# b = 0.3 # inactive rate per month

class social_network():
    def __init__(self,  alpha=0.1,beta=0.3, n = 4000,name = "ws",):
        if name == "ws":
            self.graph = nx.watts_strogatz_graph(n, k = 57, p=0.5)
        elif name == "ba":
            self.graph = nx.barabasi_albert_graph(n, m = 6)
        
        
        init_dist=np.random.choice([TRANSMISSION,IMMUNE,HEALTH],
                         size = n,p=(alpha,beta,1-alpha-beta))
        
        # list of nodes group by status
        self.healthy_list = np.where(init_dist==HEALTH)[0].tolist()
        self.transmission_list = np.where(init_dist==TRANSMISSION)[0].tolist()
        self.immmune_list = np.where(init_dist==IMMUNE)[0].tolist()
        
        for i in range(0,n):
            self.graph.node[i]['status'] = init_dist[i]

In [15]:
# initializing a social network
G = social_network(name ="ba")
g = G.graph
transmission_list = G.transmission_list
healthy_list = G.healthy_list
immmune_list = G.immmune_list
nodes = g.nodes(data = True)

In [37]:
# finish changing the transmision nodes' status
for nt in transmission_list:

    neighbors = np.array(list(g.neighbors(nt)))
    neighbors_status = np.array([g.node[neighbor]['status'] 
                        for neighbor in g.neighbors(nt)])
    # a = copy.deepcopy(neighbors_status)

    healthy_neighbors = neighbors[np.where(neighbors_status == HEALTH)[0]]
    spreaded_neighbors = healthy_neighbors[np.where(np.random.uniform(0,1,(healthy_neighbors.shape[0],)) 
                              < p1)[0]]

    if spreaded_neighbors.size!=0:    # if information has spread to the neighbors of the transmission node
        G.transmission_list.remove(nt)
        G.immmune_list.append(nt)
        
        g.node[nt]['status'] = IMMUNE
        
        for sn in spreaded_neighbors:
            G.healthy_list.remove(sn)
            G.transmission_list.append(sn)
            
            g.node[sn]['status'] = TRANSMISSION